# Image-Classification-For-Cross-Analysis-of-Chest-X-Ray

In [1]:
"""
Loads a pretrained CNN model and presents a Gradio app to the user.
The user can upload a chest X-Ray image which will be preprocessed similarly
to the trained model, then run through the model.
A prediction of 1 or more of 14 diagnoses or no diagnosis will be output.
This will be followed up with suggested treatment from an OpenAI API.
"""

# Standard imports
import pandas as pd
from pathlib import Path
import numpy as np
import requests
from PIL import Image
import tensorflow as tf
from pathlib import Path
import gradio as gr

# Gradio App Based on Aleksandar's Model

In [2]:
# first lets load our pre-trained model
xray_model = tf.keras.models.load_model(Path("Resources/detection_model.keras"))
xray_model

d:\Development\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


<Functional name=functional_1, built=True>

In [3]:
# Aleksandar's preprocessing of an image (not really used just a reference)
# Function to preprocess images (resize to desired dimensions)
def preprocess_image(img_path, target_size=(256, 256)):
    img = Image.open(img_path)
    img = img.resize(target_size)
    img = img.convert('RGB')  # Ensure all images are in RGB format
    img = np.array(img)
    return img / 255.0  # Normalize to range [0, 1]

In [9]:
# Define the function of our model that will process the image and meta data to make a prediction
def do_predict(img):
    # Process image based on preprocess code and prep to enter into model
    img = Image.fromarray(img)
    img = img.resize((256, 256))
    img = img.convert('RGB')  # Ensure all images are in RGB format
    img = np.array(img)
    img = img / 255.0 # Normalize to range [0, 1]
    img = np.array(img)
    img = np.expand_dims(img, axis=0)

    prediction = xray_model.predict(img)

    if prediction[0][0] >= 0.5:
        output = f"There is a {prediction[0][0]*100}% chance of a diagnosis. This X-ray appears clean. Disclaimer: This app should only be used by doctors as a cross reference too."
    else:
        output = "There is a suspected diagnosis with this image. Human analysis is required to determine the exact diagnosis."

    return output

In [10]:
# Create an instance of the Gradio Interface application function with parameters.
app = gr.Interface(fn=do_predict,
                   title="Chest X-Ray Cross Analysis",
                   inputs=[gr.Image(value="PIL.Image.Image")],
                   outputs=gr.Textbox(lines=20, label="Prediction", show_copy_button=True))
# Launch the app
app.launch()

Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.
